### Chloropleth maps

#### TODO:
- spatial join some stuff together? Once all set up as markers
- convert the ookla polygons to markers based centroid
- convert the cira data to shapely? format
- count up the number of test that fall within each boundary

#### Questions
- how to handle area intersections

In [2]:
ls ../data

mlab/  Municipal_Boundaries_SHP_Geographic/  ookla/  stats_can/


In [3]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

### Boundta

In [4]:
file_path = Path("../data/Municipal_Boundaries_SHP_Geographic/")

In [5]:
city_df = gpd.read_file(file_path/"CITY.shp")
hamlet_df = gpd.read_file(file_path/"HAMLETPT.shp")
indian_df = gpd.read_file(file_path/"INDIAN.shp")
octext_df = gpd.read_file(file_path/"OCTEXT.shp")

rural_df = gpd.read_file(file_path/"RURAL.shp")
svillage_df = gpd.read_file(file_path/"SVILLAGE.shp")
town_df = gpd.read_file(file_path/"TOWN.shp")
urbserv_df = gpd.read_file(file_path/"urbserv.shp")
village_df = gpd.read_file(file_path/"VILLAGE.shp")

In [6]:
print(city_df.crs, rural_df.crs)

epsg:4269 epsg:4269


In [7]:
ookla_df = gpd.read_file("../data/ookla/AB_ookla_data_2020.shp")

In [8]:
# load in the cira df as gpd, with crs?
cira = pd.read_csv("../data/cira-internet-performance.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/cira-internet-performance.csv'

In [ ]:
cira.head()

In [ ]:
cira_df = gpd.GeoDataFrame(cira, geometry=gpd.points_from_xy(cira["GEO LONGITUDE"], cira["GEO LATITUDE"]))
cira_df.drop(["GEO LONGITUDE", "GEO LATITUDE"], axis=1, inplace=True)
cira_df.{'init': 'epsg:4326'}

In [ ]:
print(cira_df.crs)

In [ ]:
cira_df.head(3)

#### Get the Centroid of Ookla polygons
- apparently i need to project to a crs where the centroid will be in accurate? Centroid calculation seems to rely on distance, so a projection that preserves distance **(Azimuthal Equidistant projection)?**
1. Change project?
2. Do calculation?

In [ ]:
ookla_df.head()

In [ ]:
print(ookla_df.crs)

In [ ]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
def get_centroid(row):
    try:
        row.geometry

In [ ]:
ookla_df.geometry.crs

In [ ]:
ookla_df.apply()

In [ ]:
ookla_df.iloc[0].geometry.

In [ ]:
ookla_df.geometry.centroid.x

In [ ]:
ookla_df["centroid"] = gpd.points_from_xy(ookla_df)

In [ ]:
ookla_df.iloc[0].geometry.centroid.x, ookla_df.iloc[0].geometry.centroid.y

In [ ]:
main_bound_df = pd.concat([rural_df, city_df])

In [ ]:
main_bound_df.head()

In [ ]:
city_df.head()

In [ ]:
rural_df.head()

### Intersections

In [ ]:
size=(24, 30)
lwidth=0.5

In [ ]:
# sliced out rural alberta

ax = rural_df.plot(figsize=size, linewidth=lwidth, color="lightblue", edgecolor="black")
#city_df.plot(figsize=size, linewidth=lwidth, color="blue", edgecolor="black", ax=ax)

### Counting Tests
- How to count up the markers that fall within a certain polygon
- Just try counting up the markers for rural_df first